In [20]:
import os
import genomepy
import pandas as pd
from torch.nn import functional as F
import torch
import numpy as np

In [21]:
data = pd.read_table("pchic/0071_Blood - Non-activated CD4+ Cells_merged_loop.txt")

In [22]:
standard_size = 5_000
def expand_enrich_pchic(df):
    df[["bait_frag_chr", "bait_frag_start", "bait_frag_end"]] = df["bait_frag"].str.split(",", expand=True)
    df[["other_frag_chr", "other_frag_start", "other_frag_end"]] = df["other_frag"].str.split(",", expand=True)
    df["bait_size"] = np.abs(df["bait_frag_start"].apply(int) - df["bait_frag_end"].apply(int))
    df["other_size"] = np.abs(df["other_frag_start"].apply(int) - df["other_frag_end"].apply(int))
    df["bait_frag_midpoint"] = df.apply(lambda x: int((int(x["bait_frag_start"]) + int(x["bait_frag_end"])) / 2), axis=1)
    df["bait_frag_standard_size_start"] = df["bait_frag_midpoint"] - standard_size
    df["bait_frag_standard_size_end"]   = df["bait_frag_midpoint"] + standard_size
    return df

In [23]:
expand_enrich_pchic(data)

,bait_frag,other_frag,N_reads,score,bait_gene,bait_gene_strand,oe_gene,oe_gene_strand,bait_closest_gene,bait_closest_gene_strand,...,bait_frag_start,bait_frag_end,other_frag_chr,other_frag_start,other_frag_end,bait_size,other_size,bait_frag_midpoint,bait_frag_standard_size_start,bait_frag_standard_size_end
0,"chr1,831895,848168","chr1,850619,874081",192,12.48,"RP11-54O7.16,RP11-54O7.1",+,"RP11-54O7.16,RP11-54O7.3,AL645608.1,SAMD11","+,-","RP11-54O7.16,RP11-54O7.1",+,...,831895,848168,chr1,850619,874081,16273,23462,840031,835031,845031
1,"chr1,831895,848168","chr1,1206874,1212438",11,5.73,"RP11-54O7.16,RP11-54O7.1",+,"UBE2J2,LINC01786","-,+","RP11-54O7.16,RP11-54O7.1",+,...,831895,848168,chr1,1206874,1212438,16273,5564,840031,835031,845031
2,"chr1,927395,936954","chr1,1206874,1212438",23,5.57,"RP11-54O7.17,HES4,ISG15","-,+","UBE2J2,LINC01786","-,+","RP11-54O7.17,HES4,ISG15","-,+",...,927395,936954,chr1,1206874,1212438,9559,5564,932174,927174,937174
3,"chr1,943677,957199","chr1,1206874,1212438",20,6.04,"ISG15,RP11-54O7.11,AGRN","+,-","UBE2J2,LINC01786","-,+","RP11-54O7.11,ISG15,AGRN","-,+",...,943677,957199,chr1,1206874,1212438,13522,5564,950438,945438,955438
4,"chr1,1005127,1034268","chr1,903641,927394",26,8.61,"RNF223,C1orf159",-,"PLEKHN1,PERM1","+,-","RNF223,C1orf159",-,...,1005127,1034268,chr1,903641,927394,29141,23753,1019697,1014697,1024697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215682,"chrY,22732050,22743996","chrY,23423342,23424529",18,7.24,"KDM5DP1,EIF1AY","-,+",NaN,NaN,"KDM5DP1,EIF1AY","-,+",...,22732050,22743996,chrY,23423342,23424529,11946,1187,22738023,22733023,22743023
215683,"chrY,22732050,22743996","chrY,23424530,23428951",15,6.34,"KDM5DP1,EIF1AY","-,+",NaN,NaN,"KDM5DP1,EIF1AY","-,+",...,22732050,22743996,chrY,23424530,23428951,11946,4421,22738023,22733023,22743023
215684,"chrY,22732050,22743996","chrY,23428952,23431276",18,7.58,"KDM5DP1,EIF1AY","-,+",NaN,NaN,"KDM5DP1,EIF1AY","-,+",...,22732050,22743996,chrY,23428952,23431276,11946,2324,22738023,22733023,22743023
215685,"chrY,22732050,22743996","chrY,23431277,23434441",14,5.51,"KDM5DP1,EIF1AY","-,+",NaN,NaN,"KDM5DP1,EIF1AY","-,+",...,22732050,22743996,chrY,23431277,23434441,11946,3164,22738023,22733023,22743023


In [14]:
genomepy.install_genome("hg19")

10:58:27 | INFO | Downloading assembly summaries from GENCODE
10:58:39 | INFO | Downloading assembly summaries from UCSC
10:58:49 | INFO | Downloading assembly summaries from Ensembl
10:59:01 | INFO | Downloading genome from UCSC. Target URL: https://hgdownload.soe.ucsc.edu/goldenPath/hg19/bigZips/chromFa.tar.gz...


Download:   0%|          | 0.00/905M [00:00<?, ?B/s]

11:00:13 | INFO | Genome download successful, starting post processing...
11:02:14 | INFO | name: hg19
11:02:14 | INFO | local name: hg19
11:02:14 | INFO | fasta: /home/kacper/.local/share/genomes/hg19/hg19.fa


Filtering Fasta: 0.00 lines [00:00, ? lines/s]

Fasta("/home/kacper/.local/share/genomes/hg19/hg19.fa")

In [24]:
hg = genomepy.Genome("hg19")

In [25]:
def fasta_to_ohe_mapping(ascii_dna):
        if ascii_dna == 65:  # A
            return 1
        if ascii_dna == 67:  # C
            return 2
        if ascii_dna == 71:  # G
            return 3
        if ascii_dna == 84:  # T
            return 4
        if ascii_dna == 97:  # a
            return 1
        if ascii_dna == 99:  # c
            return 2
        if ascii_dna == 103: # g
            return 3
        if ascii_dna == 116: # t
            return 4
        return 0 # Nn, anything else

def from_loc_to_ohe(chr, start, end):
    sequence = hg.get_seq(chr, start, end)
    buffer = torch.frombuffer(sequence.seq.encode(), dtype=torch.uint8)
    buffer = buffer.apply_(fasta_to_ohe_mapping)
    atcg_mask = buffer != 0 # 1 if actg, 0 otherwise
    buffer -= atcg_mask * 1 # offset indexing for atcg so it starts at 0 instead of 1
                            # this is needed to have the "others" be a 0000 row in OHE
    return (
        F.one_hot(buffer.long(), num_classes=4).T * atcg_mask # OHE and mask others
    )

In [36]:
n=1000
chunks = [data[i:i+n] for i in range(0,data.shape[0],n)]

In [ ]:
for j, chunk in enumerate(chunks):
    chunk_ohe = []
    for i, row in chunks[0].iterrows():
        ohe = from_loc_to_ohe(row["bait_frag_chr"], row["bait_frag_standard_size_start"], row["bait_frag_standard_size_end"])
        chunk_ohe.append(ohe)
        assert ohe.shape[1] == 10001
        print(f"Chunk {j:>4} - Row {i:>4}", end="\r")
    torch.save(torch.stack(chunk_ohe), f"pchic/intermediates/ohe_chunk_{j}.pt")